#### **Load Data**

In [32]:
with open("C:/Users/aktkr/llm-from-scratch/story.txt", "r", encoding="utf-8") as f:
    content = f.read()
    print(content)

The Perseids are raging outside again, but I am still inside, sitting on our kitchen floor with your watchmaker’s tweezers. A drop of my blood hits the cold white tile, and I suck in air through gritted teeth. And I remember.

          One year ago, we were packing a bag to watch last August’s Perseids. The full bottle slipped from your hands, shooting glass shards and wine everywhere, dyeing my jeans forever with flecks of red.

          Don’t move, you said. You’re barefoot.

          I stood in place, fidgeting, heart pounding, while you scoured every square inch of the floor. On your knees, wiry beard tucked into your shirt, running a paper towel over each individual tile. Just like you’d search for tiny wayward springs.

          You never break things, I thought. We’re going to miss the peak, I said.

          We didn’t. I was just impatient.

          The wine was gone, so we ate mushrooms instead. We lay flat on the ground until dawn, falling upward through a dizzying sta

#### **Byte-level corpus setup**

In [33]:
text = "the quick brown fox jumps over the lazy dog"
demo_corpus = [list(text.encode("utf-8"))]
print("Byte-level corpus: ", len(demo_corpus[0]))

Byte-level corpus:  43


In [34]:
corpus = [list(content.encode("utf-8"))]
print("Byte-level corpus: ", corpus)
print(len(corpus[0]))
print(max(corpus[0]), min(corpus[0]))

Byte-level corpus:  [[84, 104, 101, 32, 80, 101, 114, 115, 101, 105, 100, 115, 32, 97, 114, 101, 32, 114, 97, 103, 105, 110, 103, 32, 111, 117, 116, 115, 105, 100, 101, 32, 97, 103, 97, 105, 110, 44, 32, 98, 117, 116, 32, 73, 32, 97, 109, 32, 115, 116, 105, 108, 108, 32, 105, 110, 115, 105, 100, 101, 44, 32, 115, 105, 116, 116, 105, 110, 103, 32, 111, 110, 32, 111, 117, 114, 32, 107, 105, 116, 99, 104, 101, 110, 32, 102, 108, 111, 111, 114, 32, 119, 105, 116, 104, 32, 121, 111, 117, 114, 32, 119, 97, 116, 99, 104, 109, 97, 107, 101, 114, 226, 128, 153, 115, 32, 116, 119, 101, 101, 122, 101, 114, 115, 46, 32, 65, 32, 100, 114, 111, 112, 32, 111, 102, 32, 109, 121, 32, 98, 108, 111, 111, 100, 32, 104, 105, 116, 115, 32, 116, 104, 101, 32, 99, 111, 108, 100, 32, 119, 104, 105, 116, 101, 32, 116, 105, 108, 101, 44, 32, 97, 110, 100, 32, 73, 32, 115, 117, 99, 107, 32, 105, 110, 32, 97, 105, 114, 32, 116, 104, 114, 111, 117, 103, 104, 32, 103, 114, 105, 116, 116, 101, 100, 32, 116, 101, 101,

In [35]:
vocab = {bytes([i]).decode("latin-1") for i in range(0, 256)}

vocab_size = len(vocab) #256
vocab_size

256

#### **Counting Pairs**

In [36]:
def pair_count(corpus):
    counts = {}
    for char in corpus:
        for i in range(len(char)-1):
            pair = (char[i], char[i+1])
            counts[pair] = counts.get(pair, 0) + 1
    return counts

pair_counts = pair_count(corpus)
print(pair_counts)
print(len(pair_counts.items()))
print("Most common pair: ", max(pair_counts, key=pair_counts.get), pair_counts[max(pair_counts, key=pair_counts.get)])

{(84, 104): 6, (104, 101): 42, (101, 32): 70, (32, 80): 4, (80, 101): 4, (101, 114): 31, (114, 115): 10, (115, 101): 11, (101, 105): 6, (105, 100): 15, (100, 115): 8, (115, 32): 34, (32, 97): 36, (97, 114): 21, (114, 101): 22, (32, 114): 9, (114, 97): 3, (97, 103): 7, (103, 105): 2, (105, 110): 63, (110, 103): 24, (103, 32): 18, (32, 111): 31, (111, 117): 45, (117, 116): 9, (116, 115): 4, (115, 105): 8, (100, 101): 9, (103, 97): 4, (97, 105): 10, (110, 44): 2, (44, 32): 40, (32, 98): 22, (98, 117): 4, (116, 32): 40, (32, 73): 25, (73, 32): 23, (97, 109): 4, (109, 32): 6, (32, 115): 36, (115, 116): 13, (116, 105): 13, (105, 108): 9, (108, 108): 13, (108, 32): 14, (32, 105): 30, (110, 115): 5, (101, 44): 10, (105, 116): 26, (116, 116): 4, (111, 110): 15, (110, 32): 31, (117, 114): 22, (114, 32): 36, (32, 107): 7, (107, 105): 7, (116, 99): 8, (99, 104): 13, (101, 110): 13, (32, 102): 25, (102, 108): 6, (108, 111): 8, (111, 111): 17, (111, 114): 18, (32, 119): 35, (119, 105): 12, (116, 104

In [37]:
most_freq = sorted(pair_counts.items(), key=lambda x: -x[1])[:20]
most_freq

[((32, 32), 144),
 ((101, 32), 70),
 ((105, 110), 63),
 ((32, 116), 58),
 ((116, 104), 48),
 ((100, 32), 46),
 ((111, 117), 45),
 ((104, 101), 42),
 ((44, 32), 40),
 ((116, 32), 40),
 ((32, 97), 36),
 ((32, 115), 36),
 ((114, 32), 36),
 ((32, 119), 35),
 ((115, 32), 34),
 ((101, 114), 31),
 ((32, 111), 31),
 ((110, 32), 31),
 ((32, 105), 30),
 ((32, 121), 29)]

#### **Merge Operation**

In [38]:
def merge_pair(corpus, pair, new_token):
    new_corpus = []
    for seq in corpus:
        i = 0
        new_seq = []
        while i < len(seq):
            if i < len(seq)-1 and (seq[i], seq[i+1]) == pair:
                new_seq.append(new_token)
                i += 2
            else:
                new_seq.append(seq[i])
                i += 1
        new_corpus.append(new_seq)
    return new_corpus

In [39]:
pair = most_freq[0][0]
new_token = vocab_size
vocab_size += 1

corpus = merge_pair(corpus, pair, new_token)

In [40]:
pair_counts = pair_count(corpus)
print(sorted(pair_counts.items(), key=lambda x: -x[1])[:20])

[((101, 32), 70), ((256, 256), 64), ((105, 110), 63), ((32, 116), 58), ((116, 104), 48), ((100, 32), 46), ((111, 117), 45), ((104, 101), 42), ((44, 32), 40), ((116, 32), 40), ((32, 97), 36), ((32, 115), 36), ((114, 32), 36), ((32, 119), 35), ((115, 32), 34), ((101, 114), 31), ((32, 111), 31), ((110, 32), 31), ((32, 105), 30), ((32, 121), 29)]


In [41]:
most_freq = sorted(pair_counts.items(), key=lambda x: -x[1])[:5]
most_freq

[((101, 32), 70),
 ((256, 256), 64),
 ((105, 110), 63),
 ((32, 116), 58),
 ((116, 104), 48)]

In [44]:
for _ in range(10):
    pair = most_freq[0][0]
    new_token = vocab_size
    vocab_size += 1

    corpus = merge_pair(corpus, pair, new_token)
    pair_counts = pair_count(corpus)
    most_freq = sorted(pair_counts.items(), key=lambda x: -x[1])[:5]
    print("Final vocabulary size: ", vocab_size)
    print("Most frequent pair: ", most_freq[0])
    print(bytes([t for t in list(most_freq[0][0]) if t < 256]).decode("utf-8"))
    print("---")

Final vocabulary size:  259
Most frequent pair:  ((105, 110), 63)
in
---
Final vocabulary size:  260
Most frequent pair:  ((32, 116), 53)
 t
---
Final vocabulary size:  261
Most frequent pair:  ((111, 117), 45)
ou
---
Final vocabulary size:  262
Most frequent pair:  ((44, 32), 38)
, 
---
Final vocabulary size:  263
Most frequent pair:  ((116, 32), 38)
t 
---
Final vocabulary size:  264
Most frequent pair:  ((260, 104), 33)
h
---
Final vocabulary size:  265
Most frequent pair:  ((100, 32), 32)
d 
---
Final vocabulary size:  266
Most frequent pair:  ((101, 114), 31)
er
---
Final vocabulary size:  267
Most frequent pair:  ((115, 32), 31)
s 
---
Final vocabulary size:  268
Most frequent pair:  ((121, 261), 25)
y
---


#### **Looping BPE Steps**

In [82]:
class BPETokenizer:
    """ A minimal Byte Pair Encoding Tokenizer implemented from scratch"""
    def __init__(self, content_file: str):
        self.vocab_size = 256
        self.merges = []
        self.merge_lookup = {}
        self.reverse_lookup = {}
        with open(content_file, "r", encoding="utf-8") as f:
            self._content = f.read()

    def __pair_count(self, corpus):
        """
        Count the frequency of adjacent token pairs across all sequences in the corpus.
        Returns: dict[(int, int)] = frequency
        """
        counts = {}
        for seq in corpus:
            for i in range(len(seq)-1):
                pair = (seq[i], seq[i+1])
                counts[pair] = counts.get(pair, 0) + 1
        return counts

    def __merge_pairs(self, corpus, pair, new_token):
        """
        Merge a given pair of sequences of all the corpus.
        Returns: new corpus with merged pairs.
        """
        new_corpus = []
        for seq in corpus:
            i = 0
            new_seq = []
            while i < len(seq):
                if i < len(seq) - 1 and (seq[i], seq[i+1]) == pair:
                    new_seq.append(new_token)
                    i += 2
                else:
                    new_seq.append(seq[i])
                    i += 1
            new_corpus.append(new_seq)
        return new_corpus

    def train(self, num_merges = 1500):
        """
        Train the BPE tokenizer on the given corpus.
        """
        corpus = [list(self._content.encode("utf-8"))]

        # Count Pairs
        for _ in range(num_merges):
            counts = self.__pair_count(corpus)
            if not counts:
                break

            pair = max(counts, key=counts.get)
            new_token = self.vocab_size

            self.merges.append((pair, new_token))
            self.merge_lookup[pair] = new_token
            self.reverse_lookup[new_token] = pair

            corpus = self.__merge_pairs(corpus, pair, new_token)
            self.vocab_size += 1

        print(f"Training Complete: {len(self.merges)} merges learned.")

    def encode(self, text: str):
        """
        Encode new text using learned BPE merges.
        """
        seq = list(text.encode("utf-8"))
        for (a, b), new_tok in self.merges:
            i = 0
            new_seq = []
            while i < len(seq):
                if i < len(seq)-1 and (seq[i], seq[i+1]) == (a,b):
                    new_seq.append(new_tok)
                    i += 2
                else:
                    new_seq.append(seq[i])
                    i += 1
            seq = new_seq
        return seq

    def _expand_token(self, token):
        if token in self.reverse_lookup:
            a, b = self. reverse_lookup[token]
            return self._expand_token(a) + self._expand_token(b)
        else:
            return [token]

    def decode(self, tokens):
        """
        Decode BPE tokens back to text.
        """
        expanded = []
        for token in tokens:
            expanded.extend(self._expand_token(token))
        return bytes(expanded).decode("utf-8", errors='replace')

tokenizer = BPETokenizer("C:/Users/aktkr/llm-from-scratch/story.txt")
tokenizer.train()

encoded = tokenizer.encode("the quick brown fox jumps over the lazy dog")
print(encoded)

decoded = tokenizer.decode(encoded)
print(decoded)


Training Complete: 1318 merges learned.
[387, 257, 113, 117, 105, 334, 32, 546, 369, 110, 32, 102, 111, 120, 32, 106, 117, 109, 112, 267, 491, 275, 314, 122, 271, 434, 103]
the quick brown fox jumps over the lazy dog


#### **Visualize the token getting merged**

In [83]:
%%capture
!pip install termcolor

In [87]:
from termcolor import colored

def visualize_merge(tokenizer, num_steps=15):
    text_bytes = list(tokenizer._content.encode("utf-8"))
    corpus = [text_bytes]

    print(colored("===Initial Byte Sequence===", "cyan"))
    print(text_bytes)
    print("---")

    for _ in range(num_steps):
        print(colored(f"===Merge Step {_+1}===", "yellow"))

        counts = tokenizer._BPETokenizer__pair_count(corpus)
        if not counts:
            print("No more pairs to merge.")
            break

        top_5_pairs = sorted(counts.items(), key=lambda x: -x[1])[:5]
        print(colored("Top 5 pairs to merge:", "green"))
        for pair, freq in top_5_pairs:
            byte_seq = tokenizer.decode(list(pair))
            print(f"Pair: {pair} ('{byte_seq}'), Frequency: {freq}")

        best_pair = max(counts, key=counts.get)
        print(colored(f"Chosen pair to merge: {best_pair} ('{tokenizer.decode(list(best_pair))}')", "red"))

        new_token = tokenizer.vocab_size
        print(colored(f"New token ID assigned: {new_token}", "magenta"))

        # print(colored("Corpus before merge:", "cyan"))
        # print(corpus[0][:150], ".....")

        corpus = tokenizer._BPETokenizer__merge_pairs(corpus, best_pair, new_token)

        # print(colored("Corpus after merge:", "cyan"))
        # print(corpus[0][:150], ".....")

        tokenizer.merges.append((best_pair, new_token))
        tokenizer.merge_lookup[best_pair] = new_token
        tokenizer.reverse_lookup[new_token] = best_pair
        tokenizer.vocab_size += 1
        print("---")

    print(colored("\nVisualization Complete.", "yellow"))

In [88]:
tokenizer = BPETokenizer("C:/Users/aktkr/llm-from-scratch/story.txt")
visualize_merge(tokenizer, num_steps=5)

===Initial Byte Sequence===
[84, 104, 101, 32, 80, 101, 114, 115, 101, 105, 100, 115, 32, 97, 114, 101, 32, 114, 97, 103, 105, 110, 103, 32, 111, 117, 116, 115, 105, 100, 101, 32, 97, 103, 97, 105, 110, 44, 32, 98, 117, 116, 32, 73, 32, 97, 109, 32, 115, 116, 105, 108, 108, 32, 105, 110, 115, 105, 100, 101, 44, 32, 115, 105, 116, 116, 105, 110, 103, 32, 111, 110, 32, 111, 117, 114, 32, 107, 105, 116, 99, 104, 101, 110, 32, 102, 108, 111, 111, 114, 32, 119, 105, 116, 104, 32, 121, 111, 117, 114, 32, 119, 97, 116, 99, 104, 109, 97, 107, 101, 114, 226, 128, 153, 115, 32, 116, 119, 101, 101, 122, 101, 114, 115, 46, 32, 65, 32, 100, 114, 111, 112, 32, 111, 102, 32, 109, 121, 32, 98, 108, 111, 111, 100, 32, 104, 105, 116, 115, 32, 116, 104, 101, 32, 99, 111, 108, 100, 32, 119, 104, 105, 116, 101, 32, 116, 105, 108, 101, 44, 32, 97, 110, 100, 32, 73, 32, 115, 117, 99, 107, 32, 105, 110, 32, 97, 105, 114, 32, 116, 104, 114, 111, 117, 103, 104, 32, 103, 114, 105, 116, 116, 101, 100, 32, 116, 10